# BCDI Pipeline
### A notebook to run the `BcdiPipeline` instance

Import packages:

In [ ]:
import os
import cdiutils

#### The main parameters

In [ ]:
beamline_setup =
sample_name =
scan =
experiment_file_path =

# Choose where you wish to save the data, or leave it as it is:
dump_dir = os.getcwd() + f"/results/{sample_name}/S{scan}/"

preprocess_shape = [100, 150, 150]  # 2 or 3 values. If 2, will take the whole RC 

# voxel_reference_methods CAN BE A LIST (as long as you like) of either
# "com", "max" or a tuple of int that corresponds to the position you
# want to crop the data at (possibly 2D).
# Ex: det_reference_voxel_method = [(70, 200, 200), "com", "com"]. This
# will center a box of size preprocess_shape around voxel
# (70, 200, 200) and compute the com, recentre the box around this com,
# recompute the com (that might have changed) and recentre again.
# Particularly useful if you have aliens (com fails) or hot pixels
# (max fails), chain centring can help you find the correct com or max.
voxel_reference_methods = ["max", "com", "com"]

rocking_angle_binning = None  # Whether you want to bin in the RC direction
light_loading = False  # Load only the roi of the data defined by the voxel_reference_methods and preprocess_output_shape
hot_pixel_filter = False
background_level = None

# The Bragg reflection you measured, just to extend the d-spacing value to that of the lattice parameter
hkl = [1, 1, 1]

# Load the parameters and parse them into the BcdiPipeline class instance 
params = cdiutils.pipeline.get_params_from_variables(dir(), globals())
bcdi_pipeline = cdiutils.BcdiPipeline(params=params)

## Pre-processing

If you need to update some parameters, you can do it directly by parsing them in the `preprocess` method such as:
```
bcdi_pipeline.preprocess(
    preprocess_shape=(100, 150, 150),
    voxel_reference_methods=(80, 150, 230)
)
```

In [ ]:
bcdi_pipeline.preprocess()

## [PyNX](https://pynx.esrf.fr/en/latest/index.html) phase retrieval
See [pynx.cdi](https://pynx.esrf.fr/en/latest/modules/cdi/index.html) for documentation on the phasing algorithms used here.

Here are a few default PyNX parameters used in cdiutils:
* `support = auto`
* `support_threshold = 0.15, 0.40`
* `support_update_period = 20`
* `support_only_shrink = False`
* `support_post_expand = None` ex: "-1,1" or "-1,2-1"
* `support_update_border_n = None`
* `support_smooth_width_begin = 2`
* `support_smooth_width_end = 0.5`
* `nb_raar = 500`
* `nb_hio =  300`
* `nb_er =  200`
* `nb_run = 20`
* `nb_run_keep = 10`
* `rebin = "1, 1, 1"`  Note this must be a string !
* `positivity = False`

Note that parsing them in the `phase_retrieval` method will update them:
```
bcdi_pipeline.phase_retrieval(nb_raar=400)
```

In [ ]:
bcdi_pipeline.phase_retrieval(
    clear_former_results=True,
    nb_run=20,
    nb_run_keep=10
)

### Analyse the phasing results

In [ ]:
bcdi_pipeline.analyse_phasing_results(
    sorting_criterion="mean_to_max",
    # plot_phasing_results=False,  # Defaults to True
    # plot_phase=True,  # Defaults to False
)

### Selection of the best reconstructions and mode decomposition

In [ ]:
# Chose the number of best candidates you want to keep.
number_of_best_candidates: int = 5

# You can either select the best candidates by specifying the
# reconstruction number or just provide the number of runs you
# want to consider. The "best" runs are selected according to the
# sorting_criterion above. 
bcdi_pipeline.select_best_candidates(
    # best_runs=[10]
    nb_of_best_sorted_runs=number_of_best_candidates
)

bcdi_pipeline.mode_decomposition()

## Post-processing:
This part includes:
* orthogonalisation,
* phase manipulation such as phase unwrapping, phase ramp removal etc.
* compute the displacement field, strain, dspacing...
* plot a bunch of figures

In [ ]:
bcdi_pipeline.postprocess(isosurface=0.3, voxel_size=None, flip=False)

## Facet analysis: detection and indexing

The facet analysis requires having run the post-processing with the 
parameter `method_det_support` different from None (default behaviour). 
You can set it to `"Isosurface"` or `"Amplitude_variation"`. The latter
allows to determine the support via deriving the reconstructed
amplitude.
Ex.:
```
bcdi_pipeline.params["cdiutils"].update(
    {
        "method_det_support": "Isosurface",  
        "order_of_derivative": "Gradient",  # or "Laplacian"
        "raw_process" : True,
    }
)
```

In [ ]:

bcdi_pipeline.params.update(
    {
        "nb_facets": 26, # Expected number of facets of the particle
        "remove_edges": True,
        "nb_nghbs_min": 0,
        "authorized_index": ["max", 1],  # ['max',n],['absolute',a,b,c,...] or ['families',[h1,k1,l1],[h2,k2,l2],...]
        "index_to_display": [[1,1,1]],  # None for the top facet or [[h1,k1,l1],[h2,k2,l2],...]
        "display_f_e_c": "facet",  # 'facet', 'edge', 'corner', or 'all'
        "size": 10  # Size of the voxels in the post-processing
    }
)

bcdi_pipeline.facet_analysis()